## read in training set and build neighbour dictionary

In [1]:
file = open("train.txt","r")
raw_data = file.readlines()
file.close()

In [2]:
from collections import defaultdict
node_set = set()
in_neighbour = defaultdict(set)  # a node is followed by another nodes {key:node, values:neighbour(in set)}
out_neighbour = defaultdict(set) # a node follow another nodes, format {key:node, values:neighbour(in set)}
for instance in raw_data:  # change to full dataset
    source,sink_list = instance.split()[0],instance.split()[1:]
    node_set.add(source)
    out_neighbour[source] = set(sink_list)
    for node in sink_list:
        node_set.add(node)
        in_neighbour[node].add(source)
#print (in_neighbour)
print (len(node_set))

4867136


## Similarity calculation

In [3]:
import math

def common_neighbour(node1,node2):
    return len(in_neighbour[node1].intersection(in_neighbour[node2]))+len(out_neighbour[node1].intersection(out_neighbour[node2]))

def salton(node1,node2):
    return common_neighbour(node1,node2)/math.sqrt((len(in_neighbour[node1])+len(out_neighbour[node1]))*(len(in_neighbour[node2])+len(out_neighbour[node2])))

def Jaccard(node1,node2):
    return common_neighbour(node1,node2)/(len(in_neighbour[node1].union(in_neighbour[node2]))+len(out_neighbour[node1].union(out_neighbour[node2])))

def AA(node1,node2):
    result = 0
    intersection = in_neighbour[node1].intersection(in_neighbour[node2]).union(out_neighbour[node1].intersection(out_neighbour[node2]))
    for node in intersection:
        result += (1/math.log(len(in_neighbour[node])+len(out_neighbour[node])))
    return result

def allocation(node1,node2):
    result = 0
    common1 = in_neighbour[node1].intersection(in_neighbour[node2]).union(out_neighbour[node1].intersection(out_neighbour[node2]))
    common2 = in_neighbour[node1].intersection(out_neighbour[node2]).union(out_neighbour[node1].intersection(in_neighbour[node2]))
    common = common1.union(common2)
    
    #common = common = in_neighbour[node1].intersection(out_neighbour[node2]).union(out_neighbour[node1].intersection(in_neighbour[node2]))
    
    for node in common:
        result += 1/(len(out_neighbour[node])+len(in_neighbour[node]))
    return result


def knn(node1,node2):
    result=[]
    firts=1
    Wbin=1/math.sqrt(1+len(in_neighbour[node2]))
    Waout=1/math.sqrt(1+len(out_neighbour[node1]))
    #w2 = Waout
    #w3 = Wbin
    w4=Wbin+Waout
    w5=Wbin*Waout
    result.append(w4)
    result.append(w5)
    return result

In [4]:
print (Jaccard("1272125","2023163"))
print (salton("1272125","2023163"))
print (AA("1272125","2023163"))
print (allocation("1272125","2023163"))

0.0020502306509482316
0.012161144797036215
0.6224424629616742
0.010785645054603582


## output result for different similarity

In [26]:
import csv
fp = open("test-public.txt","r")
header = fp.readline()
test_data = fp.readlines()
fp.close()
with open('result_Jaccard.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Id","Prediction"])
    for line in test_data:
        test_id,node1,node2 = line.split()
        writer.writerow([test_id,Jaccard(node1,node2)])
csvfile.close()

In [27]:
import csv
fp = open("test-public.txt","r")
header = fp.readline()
test_data = fp.readlines()
fp.close()
with open('result_salton.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Id","Prediction"])
    for line in test_data:
        test_id,node1,node2 = line.split()
        writer.writerow([test_id,salton(node1,node2)])
csvfile.close()

In [28]:
import csv
fp = open("test-public.txt","r")
header = fp.readline()
test_data = fp.readlines()
fp.close()
with open('result_AA.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Id","Prediction"])
    for line in test_data:
        test_id,node1,node2 = line.split()
        writer.writerow([test_id,AA(node1,node2)])
csvfile.close()

In [40]:
import csv
fp = open("test-public.txt","r")
header = fp.readline()
test_data = fp.readlines()
fp.close()
with open('result_allocation.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Id","Prediction"])
    for line in test_data:
        test_id,node1,node2 = line.split()
        writer.writerow([test_id,allocation(node1,node2)])
csvfile.close()

In [14]:
sum_deg = 0
for node in node_set:
    deg = len(in_neighbour[node])+len(out_neighbour[node])
    sum_deg += deg
print (sum_deg)
    

47893204


In [15]:
def CNPA(node1,node2):
    result = 0
    sigma = 0.0000000001
    PA = (len(in_neighbour[node1])+len(out_neighbour[node1]))*(len(in_neighbour[node2])+len(out_neighbour[node2]))
    return common_neighbour(node1,node2)+(sigma*PA)/(sum_deg/len(node_set))

## New approach with new data selection method

In [5]:
sourse_set = set()
sink_set = set()
for node in out_neighbour.keys():
    if len(out_neighbour[node])!=0:
        sourse_set.add(node)
print (len(sourse_set))

19570


In [6]:
for node in in_neighbour.keys():
    if len(in_neighbour.keys())!=0 and node not in sourse_set:
        sink_set.add(node)
print (len(sink_set))

4847566


In [101]:
import random
positive_list = set()
source_list = random.sample(sourse_set,250)

In [102]:
sink_list = set()
for source in source_list:
    for sink in out_neighbour[source]:
        sink_list.add(sink)
        positive_list.add(tuple([source,sink]))
print (len(positive_list))
print (len(sink_list))

204518
149791


In [103]:
negative_list = set()
for source in source_list:
    for sink in sink_list:
        if tuple([source,sink]) not in positive_list:
            negative_list.add(tuple([source,sink]))
print (len(negative_list))

37243232


In [104]:
negative_list2 = set()
while len(negative_list2) < 100000:
    for source in source_list:
        unconnected = sink_list.difference(out_neighbour[source])
        nodes = random.sample(unconnected,5)
        for sink in nodes:
            negative_list2.add(tuple([source,sink]))
print (len(negative_list2))


101094


## label the selected training data

In [105]:
training_set = []
for node1,node2 in positive_list:
    feature = []
    feature.append(CNPA(node1,node2))
    feature.append(salton(node1,node2))
    #feature.append(AA(node1,node2))
    feature.append(allocation(node1,node2))
    feature.extend(knn(node1,node2))
    feature.append(1)
    training_set.append(feature)
for node1,node2 in negative_list2:
    feature = []
    feature.append(CNPA(node1,node2))
    feature.append(salton(node1,node2))
    #feature.append(AA(node1,node2))
    feature.append(allocation(node1,node2))
    feature.extend(knn(node1,node2))
    feature.append(0)
    training_set.append(feature)
print (len(training_set))    

305612


In [106]:
X_train = []
Y_train = []
for item in training_set:
    X_train.append(item[:len(item)-1])
    Y_train.append(item[-1])

In [107]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [50,100]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [20,50]
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
# bootstrap = [True, False]
# Create the random grid
para_grid = {'n_estimators': n_estimators,
               #'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,}

In [108]:
#from sklearn.ensemble import RandomForestRegressor
#rf = RandomForestRegressor(max_depth = 2)
#rf.fit(X_train,Y_train)

#from sklearn.ensemble import RandomForestRegressor
#rf = RandomForestRegressor(max_depth = 2)
rf = RandomForestRegressor(bootstrap = True, max_depth = 20)
grid_search = GridSearchCV(estimator = rf, param_grid = para_grid, cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
grid_search.fit(X_train, Y_train)
#rf.fit(X_train,Y_train)
best_grid = grid_search.best_estimator_
print (best_grid)
print ("train success!")

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=50 
[CV] max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=50 
[CV] max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=50 
[CV] max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV]  max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=50, total=   1.8s
[CV] max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV]  max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=50, total=  54.5s
[CV]  max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=50, total=  54.9s
[CV] max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=50 
[CV]  max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=100, total=   2.3s
[CV] max_depth=20, min_samp

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  9.9min


[CV] max_depth=20, min_samples_leaf=2, min_samples_split=10, n_estimators=100 
[CV]  max_depth=20, min_samples_leaf=2, min_samples_split=10, n_estimators=100, total= 1.8min
[CV] max_depth=20, min_samples_leaf=2, min_samples_split=10, n_estimators=100 
[CV]  max_depth=20, min_samples_leaf=2, min_samples_split=10, n_estimators=100, total= 2.1min
[CV] max_depth=20, min_samples_leaf=2, min_samples_split=10, n_estimators=100 
[CV] max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=50 
[CV] max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=50 
[CV] max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=50 
[CV]  max_depth=20, min_samples_leaf=2, min_samples_split=10, n_estimators=100, total=   3.9s
[CV] max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=100 
[CV]  max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=50, total=   2.6s
[CV] max_depth=20, min_samples_leaf=4, min_samples_split=2, n_estimators=100 
[C

[CV] max_depth=50, min_samples_leaf=2, min_samples_split=5, n_estimators=100 
[CV]  max_depth=50, min_samples_leaf=2, min_samples_split=2, n_estimators=100, total= 2.0min
[CV] max_depth=50, min_samples_leaf=2, min_samples_split=10, n_estimators=50 
[CV]  max_depth=50, min_samples_leaf=2, min_samples_split=5, n_estimators=100, total=   3.1s
[CV] max_depth=50, min_samples_leaf=2, min_samples_split=10, n_estimators=50 
[CV]  max_depth=50, min_samples_leaf=2, min_samples_split=10, n_estimators=50, total= 1.0min
[CV] max_depth=50, min_samples_leaf=2, min_samples_split=10, n_estimators=50 
[CV]  max_depth=50, min_samples_leaf=2, min_samples_split=10, n_estimators=50, total=   1.8s
[CV]  max_depth=50, min_samples_leaf=2, min_samples_split=10, n_estimators=50, total=  59.5s
[CV] max_depth=50, min_samples_leaf=2, min_samples_split=10, n_estimators=100 
[CV] max_depth=50, min_samples_leaf=2, min_samples_split=10, n_estimators=100 
[CV]  max_depth=50, min_samples_leaf=2, min_samples_split=5, n_es

[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 41.4min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
train success!


In [109]:
# predict test-public and write in file
import csv
fp = open("test-public.txt","r")
head = fp.readline()
test_file = fp.readlines()
fp.close()
print (test_file[0])
test_data = []
for line in test_file:
    Id,node1,node2 = line.split()
    feature = []
    feature.append(CNPA(node1,node2))
    feature.append(salton(node1,node2))
    #feature.append(AA(node1,node2))
    feature.append(allocation(node1,node2))
    feature.extend(knn(node1,node2))
    test_data.append(feature)
print (len(test_data))
print (test_data[0])
with open("random forest new2.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Id","Prediction"])
    for i in range(len(test_data)):
        predict = grid_search.predict([test_data[i]])[0]
        writer.writerow([i+1,predict])
csvfile.close()

1	2184483	1300190

2000
[5.640174919180601e-09, 0.0, 0, 0.6091089451179962, 0.0545544725589981]
